In [ ]:
! pip install pmdarima

In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
df = pd.read_csv('gdrive/MyDrive/Gym Dataset/gym_dataset.csv' )
df.shape

(62184, 11)

In [ ]:
df.head()

,number_people,date,timestamp,day_of_week,is_weekend,is_holiday,temperature,is_start_of_semester,is_during_semester,month,hour
0,37,2015-08-14 17:00:11-07:00,61211,4,0,0,71.76,0,0,8,17
1,45,2015-08-14 17:20:14-07:00,62414,4,0,0,71.76,0,0,8,17
2,40,2015-08-14 17:30:15-07:00,63015,4,0,0,71.76,0,0,8,17
3,44,2015-08-14 17:40:16-07:00,63616,4,0,0,71.76,0,0,8,17
4,45,2015-08-14 17:50:17-07:00,64217,4,0,0,71.76,0,0,8,17


### ARMA/ARIMA Multi Step univariate 

In [ ]:
# return a list of relative forecast lead times
def get_lead_times():
	return [1, 2, 3, 4, 5, 6]

In [ ]:
predictionSteps.clear()
testSteps.clear()

In [ ]:
predictionSteps = []
for i in range(6):
  predictionSteps.append([])

offset = 6
testSteps = []
for i in range(6):
  testSteps.append([])

X = df.number_people.values
size = 52_560
for i in range(6):
  testSteps[i] = X[size+i:int(len(X)-offset+i)]

In [ ]:
from numpy import array
from numpy import nanmedian
from statsmodels.tsa.arima.model import ARIMA
import time
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
from matplotlib import pyplot
from warnings import catch_warnings
from warnings import filterwarnings

X = df.number_people.values
size = 52_560
train, test = X[0:size], X[size:int(len(X)-offset)]
history = [x for x in train]
predictions = list()
total_size = len(test)
count = 1
for t in range(int(len(test))):
  print('current=%d, total=%d' % (count, total_size))
  count = count + 1
  # define the model
  model = ARIMA(history, order=(0,1,0))
  # return a nan forecast in case of exception
  try:
    # ignore statsmodels warnings
    with catch_warnings():
      filterwarnings("ignore")
      # fit the model
      model_fit = model.fit()
      # forecast half an hour
      yhat = model_fit.predict(len(history), len(history)+6)
      # extract lead times
      lead_times = array(get_lead_times())
      indices = lead_times - 1
      print(yhat[indices])
      for i in range(6):
        predictionSteps[i].append(yhat[i])
        #predictions.append(yhat[i])
      obs = test[t]
      #for i in range(6):
        #history.append(obs[i])
      history.append(obs)
      #print('predicted=%f, expected=%f' % (yhat, obs))
  except:
    print("exception")





In [ ]:
## Calculating Metrics 

scores = list()
scores2 = list()
# evaluate forecasts
for i in range(6):
  mae = mean_absolute_error(testSteps[i], predictionSteps[i])
  #print('MAE: %.3f' % mae)
  scores.append(mae)
  rmse = sqrt(mean_squared_error(testSteps[i], predictionSteps[i]))
  #print('RMSE: %.3f' % rmse)
  scores2.append(rmse)

In [ ]:
## Printing Metrics 

s_scores = ', '.join(['%.4f' % s for s in scores])
s_scores2 = ', '.join(['%.4f' % s for s in scores2])
print('%s:  %s' % ("MAE",  s_scores))
print('%s:  %s' % ("RMSE", s_scores2))

MAE:  2.8640, 3.4530, 3.8846, 4.2698, 4.6234, 4.9557
RMSE:  4.6339, 5.4395, 6.0139, 6.5867, 7.0748, 7.6008


#### ARIMA MODEL evaluation metrics 
* MAE: 2.8640, 3.4530, 3.8846, 4.2698, 4.6234, 4.9557
* RMSE: 4.6339, 5.4395, 6.0139, 6.5867, 7.0748, 7.6008

In [ ]:
#Overall Metrics 

# Add Each steps metric into mae_overall and rmse overall 
mae_overall = (2.8640 + 3.4530 + 3.8846 + 4.2698 + 4.6234 + 4.9557)/6
rmse_overall = (4.6339 + 5.4395 + 6.0139 + 6.5867 + 7.0748 + 7.6008)/6
#mae_overall, rmse_overall
print('MAE Overall: %.3f' % mae_overall)
print('RMSE Overall: %.3f' % rmse_overall)


MAE Overall: 4.008
RMSE Overall: 6.225


## ML linear regression model 


In [ ]:
# recursive multi-step forecast with linear algorithms
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from matplotlib import pyplot
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import Lars
from sklearn.linear_model import LassoLars
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import SGDRegressor
 
# split a univariate dataset into train/test sets
def split_dataset(data):
	# split into standard weeks
	train, test = data[:52_560], data[52_560:]
	# restructure into windows of weekly data
	train = array(split(train, int(len(train)/6)))
	test = array(split(test, len(test)/6))
	return train, test

 
# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
	scores = list()
	scores2 = list()
	# calculate an RMSE score for each day
	for i in range(actual.shape[1]):
		# calculate mse
		mse = mean_squared_error(actual[:, i], predicted[:, i])
		# calculate rmse
		rmse = sqrt(mse)
    #calculate mae

		mae = mean_absolute_error(actual[:, i], predicted[:, i])
		# store
		scores.append(rmse)
		scores2.append(mae)
	# calculate overall RMSE
	s = 0
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
			s += (actual[row, col] - predicted[row, col])**2
	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
	return score, scores, scores2
 
# summarize scores
def summarize_scores(name, score, scores, scores2):
	s_scores = ', '.join(['%.4f' % s for s in scores])
	s_scores2 = ', '.join(['%.4f' % s for s in scores2])
	print('%s: [%.3f] %s' % (name, score, s_scores))
	print('%s: %s' % (name, s_scores2))
 
# prepare a list of ml models
def get_models(models=dict()):
	# linear models
	models['lr'] = LinearRegression()
	print('Defined %d Recursive models' % len(models))
	return models
 
# create a feature preparation pipeline for a model
def make_pipeline(model):
	steps = list()
	# standardization
	steps.append(('standardize', StandardScaler()))
	# normalization
	steps.append(('normalize', MinMaxScaler()))
	# the model
	steps.append(('model', model))
	# create pipeline
	pipeline = Pipeline(steps=steps)
	return pipeline
 
# make a recursive multi-step forecast
def forecast(model, input_x, n_input):
	yhat_sequence = list()
	input_data = [x for x in input_x]
	for j in range(6):
		# prepare the input data
		X = array(input_data[-n_input:]).reshape(1, n_input)
		# make a one-step forecast
		yhat = model.predict(X)[0]
		# add to the result
		yhat_sequence.append(yhat)
		# add the prediction to the input
		input_data.append(yhat)
	return yhat_sequence
 
# convert windows of weekly multivariate data into a series of total power
def to_series(data):
	# extract just the total power from each week
	series = [week[:, 0] for week in data]
	# flatten into a single series
	series = array(series).flatten()
	return series
 
# convert history into inputs and outputs
def to_supervised(history, n_input):
	# convert history to a univariate series
	data = to_series(history)
	X, y = list(), list()
	ix_start = 0
	# step over the entire history one time step at a time
	for i in range(len(data)):
		# define the end of the input sequence
		ix_end = ix_start + n_input
		# ensure we have enough data for this instance
		if ix_end < len(data):
			X.append(data[ix_start:ix_end])
			y.append(data[ix_end])
		# move along one time step
		ix_start += 1
	return array(X), array(y)
 
# fit a model and make a forecast
def sklearn_predict(model, history, n_input):
	# prepare data
	train_x, train_y = to_supervised(history, n_input)
	# make pipeline
	pipeline = make_pipeline(model)
	# fit the model
	pipeline.fit(train_x, train_y)
	# predict the week, recursively
	yhat_sequence = forecast(pipeline, train_x[-1, :], n_input)
	return yhat_sequence
 
# evaluate a single model
def evaluate_model(model, train, test, n_input):
	# history is a list of hourly data
	history = [x for x in train]
	# walk-forward validation over each hour
	predictions = list()
	for i in range(len(test)):
		# predict the hour
		yhat_sequence = sklearn_predict(model, history, n_input)
		# store the predictions
		predictions.append(yhat_sequence)
		# get real observation and add to history for predicting the next hour
		history.append(test[i, :])
	predictions = array(predictions)
	# evaluate predictions days for each hour
	score, scores, scores2 = evaluate_forecasts(test[:, :, 0], predictions)
	return score, scores, scores2
 
df2 = df[['number_people']]
train, test = split_dataset(df2.values)
# prepare the models to evaluate
models = get_models()
n_input = 3 #prior timestamp observations
# evaluate each model

for name, model in models.items():
	# evaluate and get scores
	score, scores, scores2 = evaluate_model(model, train, test, n_input)
	# summarize scores
	summarize_scores(name, score, scores, scores2)

Defined 1 Recursive models
lr: [6.647] 5.0811, 5.7673, 6.2328, 6.9546, 7.3028, 8.0914
lr: 3.4553, 4.1442, 4.5002, 5.0973, 5.5788, 6.1500


### Linear Regression Model evaluation metrics

* MAE: 3.4553, 4.1442, 4.5002, 5.0973, 5.5788, 6.1500
* RMSE: 5.0811, 5.7673, 6.2328, 6.9546, 7.3028, 8.0914


In [ ]:
#Overall Metrics 

mae_overall = (3.4553 + 4.1442 + 4.5002 + 5.0973 + 5.5788 + 6.1500)/6
rmse_overall = 6.647
#mae_overall, rmse_overall
print('MAE Overall: %.3f' % mae_overall)
print('RMSE Overall: %.3f' % rmse_overall)

MAE Overall: 4.821
RMSE Overall: 6.647


# ML Non Linear Model



In [ ]:
# spot check nonlinear algorithms
from numpy import load
from numpy import loadtxt
from numpy import nan
from numpy import isnan
from numpy import count_nonzero
from numpy import unique
from numpy import array
from sklearn.base import clone
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor

 

In [ ]:
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from matplotlib import pyplot
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import Lars
from sklearn.linear_model import LassoLars
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import SGDRegressor
   
# split a univariate dataset into train/test sets
def split_dataset(data):
	# split into standard weeks
	train, test = data[:52_560], data[52_560:]
	# restructure into windows of weekly data
	train = array(split(train, int(len(train)/6)))
	test = array(split(test, len(test)/6))
	return train, test
 
# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
	scores = list()
	scores2 = list()
	# calculate an RMSE score for each day
	for i in range(actual.shape[1]):
		# calculate mse
		mse = mean_squared_error(actual[:, i], predicted[:, i])
		# calculate rmse
		rmse = sqrt(mse)
    #calculate mae

		mae = mean_absolute_error(actual[:, i], predicted[:, i])
		# store
		scores.append(rmse)
		scores2.append(mae)
	# calculate overall RMSE
	s = 0
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
			s += (actual[row, col] - predicted[row, col])**2
	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
	return score, scores, scores2
 
# summarize scores
def summarize_scores(name, score, scores, scores2):
	s_scores = ', '.join(['%.4f' % s for s in scores])
	s_scores2 = ', '.join(['%.4f' % s for s in scores2])
	print('%s: [%.3f] %s' % (name, score, s_scores))
	print('%s: %s' % (name, s_scores2))
 
# prepare a list of ml models
def get_models(models=dict()):
	# non-linear models
	models['knn'] = KNeighborsRegressor(n_neighbors=7)
	# # ensemble models
	n_trees = 100
	models['gbm'] = GradientBoostingRegressor(n_estimators=n_trees)
	print('Defined %d models' % len(models))
	return models
 
# create a feature preparation pipeline for a model
def make_pipeline(model):
	steps = list()
	# standardization
	steps.append(('standardize', StandardScaler()))
	# normalization
	steps.append(('normalize', MinMaxScaler()))
	# the model
	steps.append(('model', model))
	# create pipeline
	pipeline = Pipeline(steps=steps)
	return pipeline
 
# convert history into inputs and outputs
def to_supervised(history, output_ix):
	X, y = list(), list()
	# step over the entire history one time step at a time
	for i in range(len(history)-1):
		X.append(history[i][:,0])
		y.append(history[i + 1][output_ix,0])
	return array(X), array(y)
 
# fit a model and make a forecast
def sklearn_predict(model, history):
	yhat_sequence = list()
	# fit a model for each forecast day
	for i in range(6):
		# prepare data
		train_x, train_y = to_supervised(history, i)
		# make pipeline
		pipeline = make_pipeline(model)
		# fit the model
		pipeline.fit(train_x, train_y)
		# forecast
		x_input = array(train_x[-1, :]).reshape(1,6)
		yhat = pipeline.predict(x_input)[0]
		# store
		yhat_sequence.append(yhat)
	return yhat_sequence
 
# evaluate a single model
def evaluate_model(model, train, test):
	# history is a list of weekly data
	history = [x for x in train]
	# walk-forward validation over each week
	predictions = list()
	for i in range(len(test)):
		# predict the week
		yhat_sequence = sklearn_predict(model, history)
		# store the predictions
		predictions.append(yhat_sequence)
		# get real observation and add to history for predicting the next week
		history.append(test[i, :])
	predictions = array(predictions)
	# evaluate predictions days for each week
	score, scores, scores2 = evaluate_forecasts(test[:, :, 0], predictions)
	return score, scores, scores2
 

df2 = df[['number_people']]
# split into train and test
train, test = split_dataset(df2.values)
# prepare the models to evaluate
models = get_models()
# evaluate each model

for name, model in models.items():
	# evaluate and get scores
	score, scores, scores2 = evaluate_model(model, train, test)
	# summarize scores
	summarize_scores(name, score, scores, scores2)
	# plot scores



Defined 2 models
knn: [8.640] 7.8398, 8.3037, 8.4015, 8.9987, 8.9496, 9.2662
knn: 5.1589, 5.5033, 5.5837, 6.0092, 6.3286, 6.5630
gbm: [8.677] 7.8407, 8.2579, 8.4183, 8.9880, 9.0486, 9.4098
gbm: 5.1974, 5.5449, 5.6873, 6.0631, 6.4856, 6.7578


### Non Linear Regression Model evaluation metrics

* KNN
  * MAE: 5.1589, 5.5033, 5.5837, 6.0092, 6.3286, 6.5630
  * RMSE: 7.8398, 8.3037, 8.4015, 8.9987, 8.9496, 9.2662

In [ ]:
#Overall Metrics 

mae_overall = (5.1589 + 5.5033 + 5.5837 + 6.0092 + 6.3286 + 6.5630)/6
rmse_overall = 8.640
#mae_overall, rmse_overall
print('MAE Overall: %.3f' % mae_overall)
print('RMSE Overall: %.3f' % rmse_overall)

MAE Overall: 5.858
RMSE Overall: 8.640
